In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from random import randint


In [2]:
 browser = webdriver.Chrome('C:/Users/utilisateur/chromedriver/chromedriver.exe') 

In [3]:
def scrapping_lacentrale (car,pages):
    
#     15 resultat par pages
    
    #url du site lacentrale
   
    url = f'https://www.lacentrale.fr/occasion-voiture-marque-{car}.html'
    browser.get(url)
    browser.maximize_window()
    time.sleep(2)

    #creation dataframe 
    # TO DO: recuperation csv + mise a jour
    df=pd.DataFrame(columns= [item[0] for item in xpath.items()])
    

    j=0
    while j < pages:
        
        #  Recuperation de tous les href des annonces de la page
        list_link_annonces = browser.find_elements_by_xpath("//div[@class='adLineContainer']/div/a")  
        for i in range(len(list_link_annonces)):
                
                list_link_annonces[i].send_keys(Keys.CONTROL + Keys.RETURN)
                time.sleep(1)
                
                #swich sur le nouveau onglet
                browser.switch_to.window(browser.window_handles[1])
                time.sleep(1)
                print(browser.current_url)
                url_annonce=browser.current_url
                browser1=browser.get(url_annonce)
            
                # utilisation fonction infos_annonce pour recuperer les informations
                annonce={}
                
                annonce = infos_annonce(annonce, browser1)
                print(annonce)
                
                time.sleep(2)
                df= df.append(annonce, ignore_index=True)
#                 print(df)
                
                
                browser.close()
                browser.switch_to.window(browser.window_handles[0])
                
                
        
         # page suivant :    
        try:
            nextpage_button = browser.find_element_by_css_selector("[title='Page suivante']")  
            time.sleep(2)
            ActionChains(browser).move_to_element(nextpage_button) 
            time.sleep(2)
            ActionChains(browser).click(nextpage_button).perform() 
            j+=1

        except (NoSuchElementException) :
            print(":p")
            break
       
    return df

In [4]:
xpath={
"nom":"//div[@class='cbm-sideBar']/div/div/div",
"marque":"//div[@class='cbm-price']/div/div/a",
"modele":"//div[@class='cbm-price']/div/div/a",
"categorie":"//section[@class='containerSite containerContent']/div/div/div/div/div/nav/ol/li[2]/a/span",
"Emission_CO2": "//span[contains(@class, 'cbm__bilan--co2')]",
"kilometrage":"//span[preceding::button[text()[contains(.,'Kilométrage compteur')]]]",
"energie":"//span[preceding::button[text()[contains(.,'Énergie')]]]",
"boite_de_vitesse":"//span[preceding::button[text()[contains(.,'Boîte de vitesse')]]]", 
"couleur":"//span[preceding::button[text()[contains(.,'Couleur extérieure')]]]",
"nombre_portes":"//span[preceding::button[text()[contains(.,'Nombre de portes')]]]",
"nombre_place":"//span[preceding::button[text()[contains(.,'Nombre de places')]]]",
"mise_circulation":"//span[preceding::button[text()[contains(.,'Mise en circulation')]]]",
"reference_annonce":"//section[@id='generalInformation']/div[1]/span",
"conso_mixte":"//li[contains(text(),'Consommation mixte')]",
"Puissance_fiscale":"//span[preceding::button[text()[contains(.,'Puissance fiscale')]]]",
"Puissance_din":"//span[preceding::button[text()[contains(.,'Puissance din')]]]",
"Crit_Air":'//span[preceding::button[text()[contains(.,"Crit\'Air")]]]',
"prix":"//span[@class='cbm__priceWrapper']"
}




In [5]:
def get_element(key,xpath, annonce, browser1):
    
    print(browser1)
    browser1
    
    if key== 'marque':
        try:
            annonce[key]= browser.find_element_by_xpath(xpath[key]).get_attribute('data-make')
        except:
            pass 
    elif key== 'modele':
        try:
            annonce[key]= browser.find_element_by_xpath(xpath[key]).get_attribute('data-model')
        except:
            pass 
    else:    
        try:
            annonce[key]= browser.find_element_by_xpath(xpath[key]).text
        except:
            pass
    
                
    return annonce

        

def infos_annonce(annonce,browser1):
        annonce= get_element('nom',xpath,annonce,browser1)
        annonce= get_element('marque',xpath,annonce,browser1)
        annonce= get_element('modele',xpath,annonce,browser1)
        annonce= get_element('categorie',xpath,annonce,browser1)
        annonce= get_element('Emission_CO2',xpath,annonce,browser1)
        annonce= get_element('kilometrage',xpath,annonce,browser1)
        annonce= get_element('energie',xpath,annonce,browser1)
        annonce= get_element('boite_de_vitesse',xpath,annonce,browser1)
        annonce= get_element('couleur',xpath,annonce,browser1)
        annonce= get_element('nombre_portes',xpath,annonce,browser1)
        annonce= get_element('nombre_place',xpath,annonce,browser1)
        annonce= get_element('mise_circulation',xpath,annonce,browser1)
        annonce= get_element('reference_annonce',xpath,annonce,browser1)
        annonce= get_element('conso_mixte',xpath,annonce,browser1)
        annonce= get_element('Puissance_fiscale',xpath,annonce,browser1)
        annonce= get_element('Puissance_din',xpath,annonce,browser1)
        annonce= get_element('Crit_Air',xpath,annonce,browser1)
        annonce= get_element('prix',xpath,annonce,browser1)
   
        return annonce


In [12]:
# df_peugeot= scrapping_lacentrale('peugeot',1)


In [7]:
# df_renault= scrapping_lacentrale('renault',50)

In [8]:
# df_citroen=scrapping_lacentrale('citroen',50)

In [9]:
def to_csv(df):
    try:
        df.to_csv('lacentrale.csv', index=False,encoding='utf-8', mode='a')
    except:
        df.to_csv('lacentrale.csv', index=False,encoding='utf-8')


In [10]:
def clean_scrapping_centrale(df):
        df['prix'] = df['prix'].str.replace('€', '', regex=True)
        df['prix'] = df['prix'].str.replace(' ', '', regex=True)
        
        df['kilometrage'] = df['kilometrage'].str.replace('Km', '', regex=True)
        df['kilometrage'] = df['kilometrage'].str.replace(' ', '', regex=True)
        
        df['conso_mixte'] = df['conso_mixte'].str.replace('Consommation mixte :', '', regex=True)
        df['conso_mixte'] = df['conso_mixte'].str.replace('L', '', regex=True)
        df['conso_mixte'] = df['conso_mixte'].str.replace(' ', '', regex=True)
        
        df['categorie'] = df['categorie'].str.replace('occasion', '', regex=True)
        df['categorie'] = df['categorie'].str.replace(' ', '', regex=True)
        
        df['Puissance_fiscale'] = df['Puissance_fiscale'].str.replace('CV', '', regex=True)
        df['Puissance_fiscale'] = df['Puissance_fiscale'].str.replace(' ', '', regex=True)
        
        df['Puissance_din'] = df['Puissance_din'].str.replace('ch', '', regex=True)
        df['Puissance_din'] = df['Puissance_din'].str.replace(' ', '', regex=True)
        
        col_name= ['nom','marque', 'modele','categorie','co2','nb_km','carburant', 'boite_vitesse', 'couleur','nb_portes',
       'nb_places', 'mise_circulation','ref', 'conso_mixte', 'p_fiscale', 'p_din',
       'critair', 'prix']

        df.columns = col_name
        
        df= df[['nom', 'ref', 'marque', 'modele', 'categorie', 'co2',
       'nb_km', 'carburant', 'boite_vitesse', 'couleur', 'nb_portes',
       'nb_places', 'mise_circulation', 'conso_mixte', 'p_fiscale', 'p_din',
       'critair', 'prix']]
        
        for col in [ 'nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']:
                df[col] = df[col].astype(int)
        
        df.drop(df[(df.categorie =='Fourgon(moinsde3,5t)') | (df.categorie =='Fourgonnette') |
                    (df.categorie =='Break')].index, inplace=True)
        
        df.drop(df[(df.carburant =='Hybride diesel électrique') | (df.carburant =='Hybride diesel électrique') |
                    (df.carburant =='Hybride diesel électrique')].index, inplace=True)
        
        df['categorie'].replace({'4x4,SUV&Crossover':'SUV 4X4','Citadine':'CITADINE','Monospace':'MONOSPACE','Berline':'BERLINE'},inplace=True)
        df['carburant'].replace({'Diesel':'DIESEL','Essence':'ESSENCE'},inplace=True)
        df['boite_vitesse'].replace({'mécanique':'MANUELLE','automatique':'AUTO'},inplace=True)
        df['mise_circulation'] = df['mise_circulation'].str.extract('(\d{4})').astype(int)
        
        df['mise_circulation'] = df['mise_circulation'].astype(str)
        
        for i in [x for x in df.columns if x not in ['nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']]:
            df[i] = df[i].astype(str)
            df[i] = df[i].str.lower()
    
    
        for i in [x for x in df.columns if x not in ['nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']]:
            df[i] = df[i].str.capitalize()  
        
        
        return df 